In [1]:
import pandas as pd
import re

from datetime import datetime

In [2]:
month_dict = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

def interventions(text):
    pattern = r'([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:\s*[\s\S]*?)(?=[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:|\Z)'
    interventions = re.findall(pattern, text)
    return interventions


def president_interventio(text):
    match = 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR:'
    filtered_list = [elem for elem in text if elem.startswith(match)]
    return filtered_list


def convert_to_iso(date_str):
    parts = date_str.split()
    month = month_dict.get(parts[0].lower(), '01')
    day = parts[1].replace(',', '')
    year = parts[2]
    return f"{year}-{month}-{day.zfill(2)}"



## Discourses

In [3]:
json_file = '../data/raw/discourse.json'

discourses = pd.read_json(json_file)

discourses['timestamp'] = discourses['entry-date'].apply(convert_to_iso)

discourses['interventions'] = discourses['entry-content'].apply(interventions)

discourses['president'] = discourses['interventions'].apply(president_interventio)

discourses['president'] = discourses['interventions'].apply(president_interventio)

discourses = discourses[discourses['entry-title'].str.contains(
    'prensa matutina', case=False
)]

In [4]:
discourses.sample(10)

,entry-title,entry-date,entry-content,entry-url,timestamp,interventions,president
39,Versión estenográfica de la conferencia de pre...,"enero 14, 2019","2019, Año del Caudillo del Sur, Emiliano Zapat...",https://lopezobrador.org.mx/https://lopezobrad...,2019-01-14,[PRESIDENTE\nANDRÉS MANUEL LÓPEZ OBRADOR: Buen...,[]
829,Versión estenográfica de la conferencia de pre...,"agosto 13, 2020","2020, Año de Leona Vicario, Benemérita Madre d...",https://lopezobrador.org.mx/https://lopezobrad...,2020-08-13,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
2117,Versión estenográfica de la conferencia de pre...,"marzo 11, 2024","2024: Año de Felipe Carrillo Puerto, benemérit...",https://lopezobrador.org.mx/https://lopezobrad...,2024-03-11,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1211,Versión estenográfica de la conferencia de pre...,"julio 27, 2021",2021: Año de la Independencia\nPRESIDENTE ANDR...,https://lopezobrador.org.mx/https://lopezobrad...,2021-07-27,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1721,Versión estenográfica de la conferencia de pre...,"diciembre 28, 2022","2022: Año de Ricardo Flores Magón, precursor d...",https://lopezobrador.org.mx/https://lopezobrad...,2022-12-28,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1320,Versión estenográfica de la conferencia de pre...,"noviembre 4, 2021",2021: Año de la Independencia\nPRESIDENTE ANDR...,https://lopezobrador.org.mx/https://lopezobrad...,2021-11-04,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
82,Versión estenográfica de la conferencia de pre...,"febrero 7, 2019","2019, Año del Caudillo del Sur, Emiliano Zapat...",https://lopezobrador.org.mx/https://lopezobrad...,2019-02-07,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1027,Versión estenográfica de la conferencia de pre...,"enero 22, 2021",2021: Año de la Independencia\nPRESIDENTE ANDR...,https://lopezobrador.org.mx/https://lopezobrad...,2021-01-22,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1386,Versión estenográfica de la conferencia de pre...,"enero 26, 2022","2022: Año de Ricardo Flores Magón, precursor d...",https://lopezobrador.org.mx/https://lopezobrad...,2022-01-26,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...
1938,Versión estenográfica de la conferencia de pre...,"septiembre 18, 2023","2023: Año de Francisco Villa, el revolucionari...",https://lopezobrador.org.mx/https://lopezobrad...,2023-09-18,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...,[PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Bueno...


In [5]:
discourses.drop(
    columns=['entry-date', 'entry-title']
).to_json('../data/prepro/interventions.json')

## Interventions

In [6]:
interventions = discourses.explode(
    'interventions', ignore_index=True
)
interventions['speaker'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[0] if len(x.split(':', 1)) > 1 else x)
interventions['speech'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[1] if len(x.split(':', 1)) > 1 else x
)
interventions.drop(
    columns=['interventions', 'entry-date', 'president', 'entry-title', 'entry-content', 'entry-url'], 
    inplace=True
)

In [7]:
interventions.sample(10)

,timestamp,speaker,speech
51186,2021-12-21,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"El 12 de diciembre, sí, el día de Guadalupe, ..."
12651,2019-10-11,PREGUNTA,"Gracias, presidente. Alfonso Vargas, de Caden..."
35652,2021-02-15,INTERLOCUTORA,Gracias. \n¿Y en algún momento se ha pensado ...
4184,2019-04-22,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Sí, pero hay una que demuestra más, muestra m..."
102297,2024-03-12,JESÚS RAMÍREZ CUEVAS,"Sí, sí, ahora traen el agua aquí.\n"
1935,2019-01-30,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Sí, es un proceso. No, en promedio, vamos a c..."
69793,2022-11-15,INTERVENCIÓN,‘Totalmente INE’.\n
53075,2022-02-09,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Si es posible, los de la sierra, los cercanos..."
18382,2020-01-29,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Claro que sí, por convicción.\n"
95529,2023-12-05,INTERVENCIÓN,"No, progresista, de izquierda.\n"


In [8]:
interventions.to_json('../data/prepro/interventions.json')